In [ ]:
from f16lib.systems import F16AcasShield
from csaf import SystemEnv


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf = F16AcasShield()
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
def air_collision_condition(ctraces):
        """ground collision premature termination condition
        """
        # get the aircraft states
        sa, sb = ctraces['plant']['states'], ctraces['intruder_plant']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 10

In [ ]:
# create pub/sub components out of the configuration
my_system = F16AcasShield()
my_system.check()

# set the scenario states
my_system.set_state('plant', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_system.set_state('intruder_plant', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 4000.0, 0.0, 6000.0, 9.0])

In [ ]:
trajs = my_system.simulate_tspan((0, 35.0), show_status=True)

In [ ]:
plt.scatter(*np.array(trajs["plant"].states)[:, 9:11].T)
plt.scatter(*np.array(trajs["intruder_plant"].states)[:, 9:11].T)

In [ ]:
# create an environment from the system, allowing us to act as the controller
class F16AcasShieldEnv(SystemEnv):
    system_type = F16AcasShield
    agents = ["predictor"]
    
my_env = F16AcasShieldEnv(terminating_conditions_all=air_collision_condition)
# set the scenario states
my_env.set_state('plant', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_env.set_state('intruder_plant', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 4000.0, 0.0, 6000.0, 9.0])

In [ ]:
from collections import deque


class PredictorBuffer:
    # number of steps to take before re-running predictor
    n_steps = 10
    
    def __init__(self, env: SystemEnv):
        self.pstates = []
        self.init_out = [0.,0.,0.,0.7]
        self.env = env
        self._finished = False
        
    def step(self, pred_out = False):
        """step through the simulation for n steps and collect a buffer for prediction"""
        for _ in range(self.n_steps):
            try:
                # send the outputs, can collect the inputs
                ctime, comp_input = self.env.step({"outputs": [pred_out]}) 

                # get the states and track them over time
                self.pstates.append((comp_input))

            # stop iteration occurs when the termination conditions are satisfied
            except StopIteration as e:
                self._finished = True
                break

            # other errors can occur -- maybe solver error
            except Exception as e:
                self._finished = True
                raise e
                break
      
    @property
    def buffer(self) -> np.array:
        """get the buffer as a numpy array"""
        return np.array(self.pstates)
                
    @property
    def is_finished(self):
        """if simulation terminated"""
        return self._finished
    

In [ ]:
# create a buffer for the predictor
pb = PredictorBuffer(my_env)

# fill it
pb.step(False)

# get buffer
pstates = pb.buffer

In [ ]:
plt.scatter(*pstates[:, 9:11].T)
plt.scatter(*pstates[:, 9+13:11+13].T)

In [ ]:
plt.plot(pstates[:, 0])

In [ ]:
plt.scatter(*pstates[:, 9:11].T)

In [ ]:
!pip uninstall --yes numpy
!pip install numpy==1.20.0 --no-cache-dir --no-binary :all:
!pip install GPy #-U git+https://github.com/SheffieldML/GPy.git@devel

import GPy


In [ ]:
import numpy as np

In [ ]:
def prod_kernel():
    kern0 = GPy.kern.RBF(1, lengthscale=40, variance=28)
    kern1 = GPy.kern.Spline(1, c=20, variance=28)
    kern0.lengthscale.fix()
    kern = GPy.kern.Prod([kern0, kern1])
    return kern



def predict(tspan, pstates, idx=0):
    tt = (np.arange(0, len(pstates), 1) / 10)[:, np.newaxis]
    xt = (pstates[:, 9+13*idx])[:, np.newaxis]
    yt = (pstates[:, 10+13*idx])[:, np.newaxis]
    mx = GPy.models.GPRegression(tt, xt, prod_kernel(), normalizer=True)
    my = GPy.models.GPRegression(tt, yt, prod_kernel(), normalizer=True)
    mx.optimize()
    my.optimize()
    x, xv = mx.predict((np.array(tspan) + len(pstates)/10)[:, np.newaxis])
    y, yv = my.predict((np.array(tspan) + len(pstates)/10)[:, np.newaxis])
    return (tt.flatten(), xt.flatten(), yt.flatten()), (x.flatten(), xv.flatten()), (y.flatten(), yv.flatten())


alt_conf = F16AcasShield() #cconf.SystemConfig.from_toml("/csaf-system/f16_acas_shield.toml")
#comps = alt_conf.config_dict['components']
#for k, v in comps.items():
#    comps[k]['pub'] = my_conf.config_dict['components'][k]['pub'] + 15


def predict_ownship(tspan, pstates, idx=0):
    tt = (np.arange(0, len(pstates), 1) / 10)[:, np.newaxis]
    xt = (pstates[:, 9+idx*13])[:, np.newaxis]
    yt = (pstates[:, 10+idx*13])[:, np.newaxis]
    
    tr = min(tspan), max(tspan)
    
    # create pub/sub components out of the configuration
    alt_system = F16AcasShield()#csys.System.from_config(alt_conf)

    # set the scenario states
    alt_system.set_state('plant', pstates[-1, :13])
    alt_system.set_state('intruder_plant', pstates[-1, 13:])
    trajs = alt_system.simulate_tspan(tr, 
                                     show_status=False)
    
    x = np.array(trajs['plant'].states)[:, 9]
    xv = np.zeros(x.shape)
    y = np.array(trajs['plant'].states)[:, 10]
    yv = np.zeros(y.shape)
    
    return (tt.flatten(), xt.flatten(), yt.flatten()), \
            (x.flatten(), xv.flatten()), \
            (y.flatten(), yv.flatten())

In [ ]:
t = np.linspace(0, 5, 150)
(tp, xp, yp), (x, xv), (y, yv) = predict_ownship(t, pstates)

In [ ]:
def plot_results(ax, ret, color):
    (tp, xp, yp), (x, xv), (y, yv) = ret
    ax[0].plot(tp, xp, color)
    ax[0].plot(t+tp[-1]+0.01, x, color)
    ax[0].fill_between(t+tp[-1]+0.01, x+2*np.sqrt(xv), x-2*np.sqrt(xv), alpha=0.2, color=color)
    
    ax[1].plot(tp, yp, color)
    ax[1].plot(t+tp[-1]+0.01, y, color)
    ax[1].fill_between(t+tp[-1]+0.01, y+2*np.sqrt(yv), y-2*np.sqrt(yv), alpha=0.2, color=color)

In [ ]:
pb.step(False)
fig, ax = plt.subplots(figsize=(8, 10), nrows=2)
plot_results(ax, predict_ownship(t, pb.buffer, 0), 'b')
plot_results(ax, predict(t, pb.buffer, 1), 'r')


pb.step(False)
fig, ax = plt.subplots(figsize=(8, 10), nrows=2)
plot_results(ax, predict_ownship(t, pb.buffer, 0), 'b')
plot_results(ax, predict(t, pb.buffer, 1), 'r')


pb.step(False)
pb.step(False)
pb.step(False)
fig, ax = plt.subplots(figsize=(8, 10), nrows=2)
plot_results(ax, predict_ownship(t, pb.buffer, 0), 'b')
plot_results(ax, predict(t, pb.buffer, 1), 'r')

In [ ]:
plt.scatter(*pb.buffer[:, 9:11].T, s=3)
plt.scatter(*pb.buffer[:, 13+9:13+11].T, s=3)